In [1]:
import tensorflow as tf

# Enable TensorFlow 1.x compatibility mode
tf.compat.v1.disable_eager_execution()

# Path to the checkpoint (replace with your actual path)
checkpoint_path = r"G:\My Drive\Proyectos\Zebrafish Multisensory Integration\Cstart Escape Response and Integration - Behavioral Paradigm\DLC\snapshot-300000"

# Create a new graph and session in compatibility mode
graph = tf.compat.v1.Graph()
sess = tf.compat.v1.Session(graph=graph)

with graph.as_default():
    # Restore the graph
    saver = tf.compat.v1.train.import_meta_graph(checkpoint_path + '.meta')
    saver.restore(sess, checkpoint_path)
    
    # Iterate through the graph's operations and print
    for op in graph.get_operations():
        print(op.name, op.type)
    
    # Use tensorboard to write the graph for visualization
    writer = tf.compat.v1.summary.FileWriter('log_directory', sess.graph)
    writer.close()




INFO:tensorflow:Restoring parameters from G:\My Drive\Proyectos\Zebrafish Multisensory Integration\Cstart Escape Response and Integration - Behavioral Paradigm\DLC\snapshot-300000
Placeholder Placeholder
Placeholder_1 Placeholder
Placeholder_2 Placeholder
Placeholder_3 Placeholder
Placeholder_4 Placeholder
fifo_queue FIFOQueueV2
fifo_queue_enqueue QueueEnqueueV2
fifo_queue_Dequeue QueueDequeueV2
img_mean Const
sub Sub
resnet_v1_50/Pad/paddings Const
resnet_v1_50/Pad Pad
resnet_v1_50/conv1/weights/Initializer/truncated_normal/shape Const
resnet_v1_50/conv1/weights/Initializer/truncated_normal/mean Const
resnet_v1_50/conv1/weights/Initializer/truncated_normal/stddev Const
resnet_v1_50/conv1/weights/Initializer/truncated_normal/TruncatedNormal TruncatedNormal
resnet_v1_50/conv1/weights/Initializer/truncated_normal/mul Mul
resnet_v1_50/conv1/weights/Initializer/truncated_normal AddV2
resnet_v1_50/conv1/weights VarHandleOp
resnet_v1_50/conv1/weights/IsInitialized/VarIsInitializedOp VarIsI

In [ ]:
!tensorboard --logdir=log_directory

In [5]:
import json

# Create a new graph and session in compatibility mode
graph = tf.compat.v1.Graph()
sess = tf.compat.v1.Session(graph=graph)

def get_tensor_shape(tensor):
    if tensor.shape.dims is None:
        return "unknown"
    return [dim.value if dim.value is not None else "?" for dim in tensor.shape.dims]

with graph.as_default():
    # Restore the graph
    saver = tf.compat.v1.train.import_meta_graph(checkpoint_path + '.meta')
    saver.restore(sess, checkpoint_path)
    
    # Create a dictionary to store the model structure
    model_structure = {
        "layers": [],
        "variables": []
    }
    
    # Iterate through the graph's operations and collect layer information
    for op in graph.get_operations():
        if op.type in ["Conv2D", "MaxPool", "MatMul", "BiasAdd", "Relu", "Softmax"]:
            layer_info = {
                "name": op.name,
                "type": op.type,
                "input_shape": [get_tensor_shape(tensor) for tensor in op.inputs],
                "output_shape": get_tensor_shape(op.outputs[0])
            }
            model_structure["layers"].append(layer_info)
    
    # Collect variable information
    for var in tf.compat.v1.global_variables():
        var_info = {
            "name": var.name,
            "shape": get_tensor_shape(var),
            "dtype": var.dtype.name
        }
        model_structure["variables"].append(var_info)

# Print the model structure as formatted JSON
print(json.dumps(model_structure, indent=2))

# Optionally, save the model structure to a file
with open('model_structure.json', 'w') as f:
    json.dump(model_structure, f, indent=2)

print("Model structure has been saved to 'model_structure.json'")

INFO:tensorflow:Restoring parameters from G:\My Drive\Proyectos\Zebrafish Multisensory Integration\Cstart Escape Response and Integration - Behavioral Paradigm\DLC\snapshot-300000
{
  "layers": [
    {
      "name": "resnet_v1_50/conv1/Conv2D",
      "type": "Conv2D",
      "input_shape": [
        [
          1,
          "?",
          "?",
          3
        ],
        [
          7,
          7,
          3,
          64
        ]
      ],
      "output_shape": [
        1,
        "?",
        "?",
        64
      ]
    },
    {
      "name": "resnet_v1_50/conv1/Relu",
      "type": "Relu",
      "input_shape": [
        [
          1,
          "?",
          "?",
          64
        ]
      ],
      "output_shape": [
        1,
        "?",
        "?",
        64
      ]
    },
    {
      "name": "resnet_v1_50/pool1/MaxPool",
      "type": "MaxPool",
      "input_shape": [
        [
          1,
          "?",
          "?",
          64
        ]
      ],
      "output_sha

In [6]:
graph = tf.compat.v1.Graph()
sess = tf.compat.v1.Session(graph=graph)

def get_compact_shape(tensor):
    if tensor.shape.dims is None:
        return "?"
    return "x".join(str(dim.value if dim.value is not None else "?") for dim in tensor.shape.dims)

with graph.as_default():
    saver = tf.compat.v1.train.import_meta_graph(checkpoint_path + '.meta')
    saver.restore(sess, checkpoint_path)
    
    model_structure = []
    
    for op in graph.get_operations():
        if op.type in ["Conv2D", "MaxPool", "MatMul", "BiasAdd", "Relu", "Softmax"]:
            layer_info = f"{op.name.split('/')[-1]}({op.type}):{get_compact_shape(op.inputs[0])}->{get_compact_shape(op.outputs[0])}"
            model_structure.append(layer_info)
    
    var_count = len(tf.compat.v1.global_variables())
    total_params = sum(v.shape.num_elements() for v in tf.compat.v1.global_variables())
    
    model_structure.append(f"Total variables: {var_count}")
    model_structure.append(f"Total parameters: {total_params}")

compact_json = json.dumps(model_structure, separators=(',', ':'))
print(compact_json)

with open('compact_model_structure.json', 'w') as f:
    f.write(compact_json)

print("Compact model structure has been saved to 'compact_model_structure.json'")

INFO:tensorflow:Restoring parameters from G:\My Drive\Proyectos\Zebrafish Multisensory Integration\Cstart Escape Response and Integration - Behavioral Paradigm\DLC\snapshot-300000
["Conv2D(Conv2D):1x?x?x3->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","MaxPool(MaxPool):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x256","Conv2D(Conv2D):1x?x?x64->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x256","Relu(Relu):1x?x?x256->1x?x?x256","Conv2D(Conv2D):1x?x?x256->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x256","Relu(Relu):1x?x?x256->1x?x?x256","MaxPool(MaxPool):1x?x?x256->1x?x?x256","Conv2D(Conv2D):1x?x?x256->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x64","Relu(Relu):1x?x?x64->1x?x?x64","Conv2D(Conv2D):1x?x?x64->1x?x?x256","Relu(Relu):1x?x?x256->1x?x?x256","Conv2D(Conv2D